# LangGraph Integration with Nova 2 Omni - Stateful Multimodal Reasoning

This notebook demonstrates using Amazon Nova 2 Omni with LangGraph for stateful workflows. We use direct boto3 calls for reasoning configuration and LangGraph for state management.

**Key Features:**
- Stateful workflow management with LangGraph
- Direct boto3 calls with reasoning configuration
- Multi-step reasoning with state persistence
- Conditional routing based on outputs

---

## Setup and Installation

In [ ]:
!pip install langgraph langchain-core -q

In [ ]:
import json
import boto3
from typing import Annotated, Literal, TypedDict
from botocore.config import Config
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langchain_core.tools import tool

import nova_utils

MODEL_ID = "us.amazon.nova-2-omni-v1:0"
REGION_ID = "us-west-2"

def get_bedrock_runtime():
    config = Config(read_timeout=2 * 60)
    return boto3.client(
        service_name="bedrock-runtime",
        region_name=REGION_ID,
        config=config
    )

## Define State and Tools

Create state schema and tool definitions for the workflow.

In [ ]:
class ReasoningState(TypedDict):
    """State for multimodal reasoning workflow."""
    messages: list
    analysis_complete: bool
    final_answer: str

class VideoSegmentInput(BaseModel):
    """Schema for video segment analysis."""
    timestamp_range: str = Field(description="Time range in format MM:SS-MM:SS")
    action_description: str = Field(description="What action is happening")
    key_objects: list[str] = Field(description="Key objects visible in segment")

@tool(args_schema=VideoSegmentInput)
def log_video_segment(timestamp_range: str, action_description: str, key_objects: list[str]) -> dict:
    """Log analysis of a video segment with timestamp and details."""
    return {
        "status": "segment_logged",
        "timestamp": timestamp_range,
        "action": action_description,
        "objects": key_objects
    }

class VideoSummaryInput(BaseModel):
    """Schema for complete video summary."""
    title: str = Field(description="Title or main topic of video")
    total_segments: int = Field(description="Number of segments analyzed")
    key_takeaways: list[str] = Field(description="Main takeaways from video")

@tool(args_schema=VideoSummaryInput)
def submit_video_summary(title: str, total_segments: int, key_takeaways: list[str]) -> dict:
    """Submit final video summary after analyzing all segments."""
    return {
        "status": "summary_complete",
        "title": title,
        "segments": total_segments,
        "takeaways": key_takeaways
    }

def langchain_tool_to_bedrock(lc_tool):
    schema = lc_tool.args_schema.model_json_schema()
    return {
        "toolSpec": {
            "name": lc_tool.name,
            "description": lc_tool.description,
            "inputSchema": {"json": schema}
        }
    }

bedrock_tools = [langchain_tool_to_bedrock(log_video_segment), langchain_tool_to_bedrock(submit_video_summary)]

## Build the Reasoning Graph

Create a stateful graph with reasoning nodes.

In [ ]:
bedrock = get_bedrock_runtime()

def reasoning_node(state: ReasoningState):
    """Main reasoning node using direct boto3 calls."""
    # Only use the first user message for reasoning (no assistant prefill)
    user_messages = [msg for msg in state["messages"] if msg["role"] == "user"]
    request = {
        "modelId": MODEL_ID,
        "messages": user_messages,
        "toolConfig": {"tools": bedrock_tools},
        "additionalModelRequestFields": {
            "reasoningConfig": {
                "type": "enabled",
                "maxReasoningEffort": "medium"
            }
        }
    }
    
    response = bedrock.converse(**request)
    
    # Extract content, filtering out reasoningContent blocks
    content = []
    for item in response["output"]["message"]["content"]:
        if "reasoningContent" not in item:
            content.append(item)
    
    new_message = {
        "role": "assistant",
        "content": content
    }
    
    return {"messages": state["messages"] + [new_message]}

def should_continue(state: ReasoningState) -> str:
    """Determine if workflow should continue."""
    last_message = state["messages"][-1]
    
    for content in last_message.get("content", []):
        if "toolUse" in content:
            if content["toolUse"]["name"] == "submit_video_summary":
                return "end"
    return "continue"

workflow = StateGraph(ReasoningState)
workflow.add_node("reasoning", reasoning_node)
workflow.set_entry_point("reasoning")
workflow.add_conditional_edges(
    "reasoning",
    should_continue,
    {"continue": "reasoning", "end": END}
)

app = workflow.compile()
print("✅ Reasoning graph compiled")

---

## Example: Video Analysis Workflow

Run a stateful reasoning workflow for video analysis.

In [ ]:
# Load image
image_path = "media/man_crossing_street.png"
image_bytes, image_format = nova_utils.load_image_as_bytes(image_path)

initial_state = {
    "messages": [
        {
            "role": "user",
            "content": [
                {"image": {"format": image_format, "source": {"bytes": image_bytes}}},
                {"text": "Analyze this image and provide a summary. Use submit_video_summary with a title describing the scene, number of key elements (3-5), and key observations."}
            ]
        }
    ],
    "analysis_complete": False,
    "final_answer": ""
}

print("=== Running Workflow ===")
final_state = app.invoke(initial_state)

print("\n=== Final Answer ===")
for message in final_state["messages"]:
    if message.get("role") == "assistant":
        for content in message.get("content", []):
            if "toolUse" in content:
                tool_use = content["toolUse"]
                print(f"Tool: {tool_use['name']}")
                print(json.dumps(tool_use["input"], indent=2))

---

## Key Takeaways

- **Stateful Workflows**: LangGraph maintains state across reasoning steps
- **Direct boto3 Calls**: Enable reasoning configuration with additionalModelRequestFields
- **Conditional Routing**: Route based on tool calls and outputs
- **Hybrid Approach**: Combine LangGraph structure with boto3 API control

## Next Steps

- Explore **06_strands_multimodal_reasoning.ipynb** for multi-agent patterns
- Experiment with different reasoning effort levels
- Build custom workflows for your use cases